<a href="https://colab.research.google.com/github/chrisagon/collab_python/blob/master/Directives_de_base_du_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lignes directrices pour les messages-guides
Dans cette leçon, vous mettrez en pratique deux principes de guidage et leurs tactiques connexes afin de rédiger des messages-guides efficaces pour les grands modèles de langue.
> Nous allons traduire *prompting* par *guidage* puisqu'il s'agit de guider l'I.A. dans ce que nous lui demandons et *prompt* par *message-guide*. 

## Configuration
#### Chargez la clé API et les bibliothèques Python appropriées.

Dans ce cours, nous avons fourni du code qui charge la clé API OpenAI pour vous.

In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import openai
openai.api_key = 'sk-nCBFfyMTFpnWcX0uxxxxxxxxxdJsGRdVqn2mX9exxx9'
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# openai.api_key  = os.getenv('OPENAI_API_KEY')

#### fonction d'aide
Tout au long de ce cours, nous utiliserons le modèle `gpt-3.5-turbo` d'OpenAI et le [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

Cette fonction d'aide `get_completion` facilitera l'utilisation des invites et l'examen des résultats générés :

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # c'est le degré d'aléa de la sortie du modèle
    )
    return response.choices[0].message["content"]

## Principes de guidage
- Principe 1 : Rédiger des instructions claires et précises**.
- Principe 2 : Donnez au modèle le temps de "réfléchir "**.

### Tactiques

#### Tactique 1 : Utilisez des délimiteurs pour indiquer clairement les parties distinctes de l'entrée.
- Les délimiteurs peuvent être n'importe quoi comme : ```, """, < >, `<tag> </tag>`, `:`

In [5]:
text = f"""
Vous devez exprimer ce que vous attendez d'un modèle en \ 
fournissant des instructions aussi claires et spécifiques que possible. \ 
Cela guidera le modèle vers le résultat souhaité, \
et réduira les risques de recevoir des informations non pertinentes ou incorrectes. Ne confondez pas la rédaction d'une \
 invite claire avec la rédaction d'une invite courte. \ 
Dans de nombreux cas, les invites plus longues offrent plus de clarté \ 
et de contexte pour le modèle, ce qui peut conduire à une meilleure compréhension \
de la part du modèle et réduire les risques de recevoir des réponses non pertinentes \ ou incorrectes, et \
des résultats plus détaillés et plus pertinents. 
"""
prompt = f"""
Résumez le texte délimité par des triples crochets \
en une seule phrase.
```{text}```
"""
response = get_completion(prompt)
print(response)

Pour obtenir des résultats pertinents et éviter les informations incorrectes, il est important de fournir des instructions claires et spécifiques aux modèles, même si cela signifie des invites plus longues pour offrir plus de contexte.


#### Tactique 2 : demander un résultat structuré
- JSON, HTML

In [ ]:
prompt = f"""
Créez une liste de trois titres de livres inventés \
avec leurs auteurs et leurs genres. 
Fournissez-les au format JSON avec les clés suivantes : 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

{
    "book_id": 1,
    "title": "Le Secret de la Forêt",
    "author": "Marie Dupont",
    "genre": "Fantasy"
},
{
    "book_id": 2,
    "title": "Le Mystère de la Maison Hantée",
    "author": "Jeanne Martin",
    "genre": "Thriller"
},
{
    "book_id": 3,
    "title": "L'Amour en Temps de Guerre",
    "author": "Lucie Dubois",
    "genre": "Romance Historique"
}


#### Tactique 3 : Demander au modèle de vérifier si les conditions sont remplies

In [6]:
texte_1 = f"""
Préparer une tasse de thé, c'est facile ! Tout d'abord, il faut faire bouillir de l'eau. 
de l'eau bouillante. Pendant ce temps, \
prends une tasse et mets-y un sachet de thé. Une fois que l'eau est \
suffisamment chaude, verse-la sur le sachet de thé. \ 
Laisse reposer un peu pour que le thé puisse infuser. Au bout de quelques minutes, retirez le sachet de thé. 
quelques minutes, retirez le sachet de thé. 
Si vous le souhaitez, vous pouvez ajouter du sucre ou du lait. \ 
Et le tour est joué ! Vous avez une délicieuse \
tasse de thé à déguster. 
"""
prompt = f"""
Vous obtenez un texte délimité par des guillemets triples. 
S'il contient une séquence d'instructions, \
réécrivez ces instructions dans le format suivant :

Étape 1 - ...
Étape 2 - ...
...
Étape N - ...

Si le texte ne contient pas de séquence d'instructions,\
il suffit alors d'écrire \"Aucune étape n'est fournie\".

\"\"\"{texte_1}\"\"\"
"""
response = get_completion(prompt)
print("Complétion pour le texte 1 :")
print(response)

Complétion pour le texte 1 :
Étape 1 - Faire bouillir de l'eau.
Étape 2 - Prendre une tasse et y mettre un sachet de thé.
Étape 3 - Verser l'eau bouillante sur le sachet de thé.
Étape 4 - Laisser infuser le thé pendant quelques minutes.
Étape 5 - Retirer le sachet de thé.
Étape 6 - Ajouter du sucre ou du lait (facultatif).
Étape 7 - Déguster votre délicieuse tasse de thé.

Note : Ce texte contient une séquence d'instructions.


In [7]:
texte_2 = f"""
Le soleil brille aujourd'hui et les oiseaux chantent.
chantent. C'est une belle journée pour faire une \
promenade dans le parc. 
Les fleurs s'épanouissent et les \
arbres se balancent doucement dans la brise. Les gens \
sont dehors et profitent du beau temps. \ 
Certains pique-niquent, d'autres jouent \
à des jeux ou se détendent simplement sur l'herbe. C'est une \
journée parfaite pour passer du temps à l'extérieur et apprécier la \
beauté de la nature. 
"""
prompt = f"""
Vous recevrez un texte délimité par des guillemets triples. 
S'il contient une séquence d'instructions, \
réécrivez ces instructions dans le format suivant :

Étape 1 - ...
Étape 2 - ...
...
Étape N - ...

Si le texte ne contient pas de séquence d'instructions,  
il suffit alors d'écrire \"Aucune étape n'est fournie\".

\"\"\"{texte_2}\"\"\"
"""
response = get_completion(prompt)
print("Complétion pour le texte 2 :")
print(response)

Complétion pour le texte 2 :
Aucune étape n'est fournie.


#### Tactique 4 : guidage "à quelques coups".


In [9]:
invite = f"""
Votre tâche consiste à répondre dans un style cohérent.

<enfant> : Apprenez-moi la patience.

<grandparent> : La rivière qui creuse la vallée la plus profonde \
est née d'une source modeste; la \
symphonie la plus grandiose naît d'une seule note ; \
la tapisserie la plus complexe commence par un fil solitaire.

<Enfant> : Apprenez-moi ce qu'est la résilience.
"""
response = get_completion(invite)
print(response)

<Grandparent> : La résilience est la capacité de faire face aux défis et aux difficultés de la vie, de rebondir après des épreuves et de continuer à avancer malgré les obstacles. C'est comme un arbre qui plie sous le vent mais qui ne se brise pas, ou comme une fleur qui pousse à travers les fissures du béton. La résilience est une qualité précieuse qui peut être cultivée en faisant preuve de courage, de persévérance et de confiance en soi.


### Principe 2 : Donner au modèle le temps de "penser" 

#### Tactique 1 : Spécifier les étapes nécessaires à la réalisation d'une tâche

In [ ]:
text = f"""
Dans un charmant village, les frères et sœurs Jack et Jill partent \
dans une quête pour aller chercher de l'eau dans un \
puits situé au sommet d'une colline. 
Alors qu'ils grimpent en chantant joyeusement, le malheur \
frappe - Jack trébuche sur une pierre et tombe en bas \
de la colline, suivi par Jill. \ 
Bien que légèrement blessés, les deux hommes rentrent à la maison et \
se retrouvent dans des étreintes réconfortantes. 
Malgré cette mésaventure, \
leur esprit d'aventure est resté intact. \
et ils ont continué à explorer avec plaisir.
"""
# exemple 1
prompt_1 = f"""
Effectuez les actions suivantes : 
1 - Résumez en 1 phrase le texte suivant délimité par des triples \N
avec 1 phrase.
2 - Traduire le résumé en anglais.
3 - Lister chaque nom dans le résumé en anglais.
4 - Produire un objet json contenant les éléments suivants
clés : english_summary, num_names.

Séparez vos réponses par des sauts de ligne.

Le texte :
```{text}```
"""
response = get_completion(prompt_1)
print("\nRéponse à l'invite 1 :")
print(response)

#### Demande de sortie dans un format spécifié

In [ ]:
prompt_2 = f"""
Votre tâche consiste à effectuer les actions suivantes : 
1 - Résumez le texte suivant délimité par 
  <> en 1 phrase.
2 - Traduisez le résumé en français.
3 - Lister chaque nom dans le résumé en français.
4 - Produire un objet json contenant les clés suivantes 
  clés suivantes : french_summary, num_names.

Utilisez le format suivant :
Texte : <texte à résumer>
Résumé : <résumé>
Traduction : <summary translation>
Noms : <liste de noms dans le résumé italien>
JSON de sortie : <json avec résumé et num_names>

Texte : <{texte}>
"""
response = get_completion(prompt_2)
print("\NComplétion pour l'invite 2 :")
print(response)

#### Tactique 2 : Demander au modèle d'élaborer sa propre solution avant de se précipiter sur une conclusion.